## Ignore Warning

In [23]:
import warnings
warnings.filterwarnings('ignore')

## Import API

In [24]:
from utils import load_env
load_env()
from utils import get_tavily_api_key
TAVILY_API_KEY = get_tavily_api_key()

## Import Function

In [25]:
from utils import llama32
from utils import llama31
from utils import cprint
from utils import disp_image
import json

## User Interface

In [26]:
import gradio as gr
from PIL import Image
import os

# Define the folder where your images are stored
IMAGE_FOLDER = "images"

# Function to process the uploaded image
def process_image(image):
    # Save the uploaded image to the "images" folder
    image_path = os.path.join(IMAGE_FOLDER, "uploaded_image.jpg")
    image.save(image_path)
    
    # Open and return the image
    img = Image.open(image_path)
    return img

# Create Gradio interface
interface = gr.Interface(
    fn=process_image,              # The function to process the image
    inputs=gr.Image(type="pil"),   # Accept image input in PIL format
    outputs=gr.Image(type="pil"),  # Display the output image
    title="Upload and Process Image",
    description="Upload an image and it will be saved in the images folder."
)

# Launch the interface in the notebook
interface.launch(share=True, inline=True)


Running on local URL:  http://127.0.0.1:7866


INFO:httpx:HTTP Request: GET http://127.0.0.1:7866/startup-events "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: HEAD http://127.0.0.1:7866/ "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://api.gradio.app/v2/tunnel-request "HTTP/1.1 200 OK"


Running on public URL: https://7a08e6656f7e164eba.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


INFO:httpx:HTTP Request: HEAD https://7a08e6656f7e164eba.gradio.live "HTTP/1.1 200 OK"


In [27]:
import base64

def encode_image(relative_image_path):
    # Get the absolute path relative to the notebook location
    base_path = os.getcwd()  # Get current working directory
    image_path = os.path.join(base_path, relative_image_path)

    # Encode the image
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Provide the relative path to the image
relative_path = "images/uploaded_image.jpg"
base64_image = encode_image(relative_path)


## Prompt Engineering

In [28]:
def llama32pi(prompt, image_url, model_size=11):
  messages = [
    {
      "role": "user",
      "content": [
        {"type": "text",
          "text": prompt},
        {"type": "image_url",
          "image_url": {
            "url": image_url}
        }
      ]
    },
  ]

  result = llama32(messages, model_size)
  return result

In [29]:
question = ("As a professional nutritionist and food detection expert, analyze the image provided of the interior of a refrigerator.You are analyzing the contents of a typical household refrigerator to assist in meal planning, grocery shopping, or nutritional advice. The goal is to provide a clear, concise, and comprehensive inventory of the food items present. Identify and list all visible food ingredients in the refrigerator, specifying details such as the type of food (e.g., fruits, vegetables, dairy, proteins), packaging (e.g., jar, container, loose), and the condition or freshness of each item if possible. Prioritize accuracy and clarity in your response.")

In [30]:
base64_image = encode_image("images/uploaded_image.jpg")
result = llama32pi(question, f"data:image/jpg;base64,{base64_image}")
print(result)

The refrigerator contains a variety of food items, including:

**Dairy and Eggs**

* A container of Becel margarine
* A container of yogurt
* A carton of milk
* A dozen eggs

**Meat and Poultry**

* A package of ground beef
* A package of chicken breasts
* A package of pork chops

**Fruits and Vegetables**

* A head of lettuce
* A bunch of celery
* A bag of apples
* A bag of oranges
* A bag of grapes
* A bag of carrots
* A bag of broccoli

**Beverages**

* A bottle of juice
* A can of soda
* A carton of juice
* A bottle of water

**Condiments and Sauces**

* A jar of mayonnaise
* A jar of ketchup
* A bottle of hot sauce
* A bottle of soy sauce

**Miscellaneous**

* A bag of ice
* A box of cereal
* A bag of coffee
* A box of tea

Overall, the refrigerator appears to be well-stocked with a variety of food items, including dairy products, meats, fruits, vegetables, beverages, condiments, and miscellaneous items. The condition and freshness of each item are not explicitly stated, but based

## Search API

In [31]:
import gradio as gr
from tavily import TavilyClient
tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

# Define function to use Tavily API to search for food-related information
def search_food_information(food_item):
    search_result = tavily_client.search("Describe nutrition of" + food_item)
    return search_result

# Gradio interface to take input and display the result
search_interface = gr.Interface(
    fn=search_food_information,
    inputs="text",
    outputs="json",
    title="Food Information Search",
    description="Enter a food item to get more information using Tavily."
)

# Launch the interface in the notebook
search_interface.launch(share=True, inline=True)


Running on local URL:  http://127.0.0.1:7867


INFO:httpx:HTTP Request: GET http://127.0.0.1:7867/startup-events "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: HEAD http://127.0.0.1:7867/ "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://api.gradio.app/v2/tunnel-request "HTTP/1.1 200 OK"


Running on public URL: https://395b8b88c3ecc3cffa.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


INFO:httpx:HTTP Request: HEAD https://395b8b88c3ecc3cffa.gradio.live "HTTP/1.1 200 OK"


## Put it all together.

In [34]:
import gradio as gr
import os
import base64
import logging
from tavily import TavilyClient

# Set up logging
logging.basicConfig(level=logging.INFO)

# Load environment variables
load_env()
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

# Ensure the API key is set
if not TAVILY_API_KEY:
    raise ValueError("Tavily API key is missing. Please ensure it is correctly set.")

# Initialize Tavily client
tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

# Define the folder where your images are stored
IMAGE_FOLDER = "images"
if not os.path.exists(IMAGE_FOLDER):
    os.makedirs(IMAGE_FOLDER)

# Function to process the uploaded image
def process_image(image):
    try:
        # Save the uploaded image to the "images" folder
        image_path = os.path.join(IMAGE_FOLDER, "uploaded_image.jpg")
        image.save(image_path)

        # Encode the image to base64
        base64_image = encode_image(image_path)
        image_data_url = f"data:image/jpeg;base64,{base64_image}"
        logging.info(f"Encoded image length: {len(base64_image)}")

        # Define the question to be asked
        question = ("As a professional nutritionist and food detection expert, analyze the image provided of the interior of a refrigerator.You are analyzing the contents of a typical household refrigerator to assist in meal planning, grocery shopping, or nutritional advice. The goal is to provide a clear, concise, and comprehensive inventory of the food items present. Identify and list all visible food ingredients in the refrigerator, specifying details such as the type of food (e.g., fruits, vegetables, dairy, proteins), packaging (e.g., jar, container, loose), and the condition or freshness of each item if possible. Prioritize accuracy and clarity in your response.")


        # Get the analysis result from llama32pi
        result = llama32pi(question, image_data_url)
        return result

    except FileNotFoundError as e:
        return f"Error: {e}"
    except Exception as e:
        return f"An unexpected error occurred: {e}"

# Function to use Tavily API to search for food-related information
def search_food_information(food_item):
    # Validate the input
    if not food_item.strip():
        return "Please enter a valid food item."

    try:
        # Call Tavily API
        search_result = tavily_client.search("Describe nutrition of " + food_item)
        # Format the search result for better readability
        if not search_result.get('results'):
            return "No results found for the given food item."

        formatted_result = f"Query: {search_result['query']}\n\n"
        for idx, result in enumerate(search_result['results'], start=1):
            formatted_result += (f"Result {idx}:\n"
                                f"Title: {result['title']}\n"
                                f"URL: {result['url']}\n"
                                f"Content: {result['content']}\n\n")
        return formatted_result
    except Exception as e:
        return f"An error occurred while searching for information: {e}"

# Create a combined Gradio interface for uploading image and searching for nutrition information
with gr.Blocks() as demo:
    gr.Markdown("# Know Your Nutrients - Food Analysis and Nutrition Search")
    
    # Upload and process the image section
    with gr.Row():
        with gr.Column():
            image_input = gr.Image(type="pil", label="Upload Refrigerator Image")
            analyze_button = gr.Button("Analyze Image")
        with gr.Column():
            analysis_output = gr.Textbox(label="Refrigerator Analysis Result")

    # Image analysis function
    analyze_button.click(fn=process_image, inputs=image_input, outputs=analysis_output)
    
    # Nutrition search section
    with gr.Row():
        with gr.Column():
            food_input = gr.Textbox(label="Enter Food Item to Search Nutrition Information")
            search_button = gr.Button("Search Nutrition")
        with gr.Column():
            search_output = gr.Textbox(label="Nutrition Information Result")

    # Food information search function
    search_button.click(fn=search_food_information, inputs=food_input, outputs=search_output)

# Launch the interface in the notebook
demo.launch(share=True, inline=True)


Running on local URL:  http://127.0.0.1:7870


INFO:httpx:HTTP Request: GET http://127.0.0.1:7870/startup-events "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: HEAD http://127.0.0.1:7870/ "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://api.gradio.app/v2/tunnel-request "HTTP/1.1 200 OK"


Running on public URL: https://b41af9d35213e1b439.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


INFO:httpx:HTTP Request: HEAD https://b41af9d35213e1b439.gradio.live "HTTP/1.1 200 OK"


INFO:root:Encoded image length: 295464
